## Fetch Data form Database

In [178]:
import pandas as pd
import numpy as np
import ta
import pymongo
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.subplots as sp
import yfinance as yf
# Fetch data from database

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
# Select the database
db = client["streaming_data"]
# Select the collection
collection = db["5m_stock_datastream"]
# Fetch specific stock
data = collection.find({'symbol': 'ISRG'})
# Convert to pandas dataframe
df = pd.DataFrame(list(data))

# Set maximum number of columns to display
pd.set_option('display.max_columns', None)  
# Set maximum width for each column
pd.set_option('display.max_colwidth', None)  
# Set maximum width for the entire DataFrame
pd.set_option('display.width', None)  

In [132]:
df

,date,symbol,_id,interval,bullish_engulfer,datetime,bearish_engulfer
0,2024-09-30 13:30:00,ISRG,66fb293abaf3e7bfc7be238e,60m,True,2024-09-30 13:30:00,NaN
1,2024-09-30 14:00:00,ISRG,66fb293abaf3e7bfc7be238f,60m,NaN,2024-09-30 14:00:00,True
2,2024-09-30 15:00:00,ISRG,66fb293abaf3e7bfc7be2390,60m,True,2024-09-30 15:00:00,NaN


## Add Technical Features

In [33]:
# Add bull and bear features in the dataframe
class add_features:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["open"] - self.df["close"])
        self.df["CandleStickType"] = np.where(self.df["open"] < self.df["close"], "green", "red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"close_t-{i}"] = self.df["close"].shift(i)

        self.df['Incremental_High'] = (self.df['close'] > self.df['close_t-1']) \
                                        & (self.df['close_t-1'] > self.df['close_t-2']) \
                                        & (self.df['close_t-2'] > self.df['close_t-3'])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df

    def add_ema_band(self, threshold=0.05):
        self.df['169EMA_Upper'] = self.df['169EMA'] * (1 + threshold)
        self.df['169EMA_Lower'] = self.df['169EMA'] * (1 - threshold)
        return self.df
    
    # Add Technical Indicators
    def add_technical(self):
        
        # Add ema dual channels technical indicators
        self.df['8EMA'] = ta.trend.ema_indicator(self.df['close'], window=8)
        self.df['13EMA'] = ta.trend.ema_indicator(self.df['close'], window=13)
        self.df['144EMA'] = ta.trend.ema_indicator(self.df['close'], window=144)
        self.df['169EMA'] = ta.trend.ema_indicator(self.df['close'], window=169)

        # Add MACD technical indicator
        self.df['MACD'] = ta.trend.macd(self.df['close'], 
                                        window_slow=26, 
                                        window_fast=12)
        
        self.df['MACD_SIGNAL'] = ta.trend.macd_signal(self.df['close'], 
                                                    window_slow=26, 
                                                    window_fast=12, 
                                                    window_sign=9)
        
        self.df['MACD_HIST'] = ta.trend.macd_diff(self.df['close'], 
                                                window_slow=26, 
                                                window_fast=12, 
                                                window_sign=9)
        
        # Add ATR technical indicator
        self.df["atr"] = ta.volatility.AverageTrueRange(high=self.df.high, 
                                                        low=self.df.low, 
                                                        close=self.df.close).average_true_range()
        
        self.df["atr"] = self.df.atr.rolling(window=30).mean()
        
        return self.df
    
    def apply(self):
        self.add_technical()
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        
        # Use last 1000 rows of the dataframe
        self.df = self.df.tail(3000) if len(self.df) > 3000 else self.df
        return self.df
    
testing_df = add_features(df).apply()

## Apply Strategy

In [34]:
class Alert:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['open'].shift(1)
        prev_close = self.df['close'].shift(1)
        
        # Current Candle
        current_open = self.df['open']
        current_close = self.df['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['high'] - self.df['low']
        fib_382_level = self.df['high'] - 0.382 * diff
        open_above_fib_382 = self.df['open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def add_alert(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df
    
# Apply the strategy
testing_df = Alert(testing_df).add_alert()

print(f"Bullish MACD Count: {len(testing_df[testing_df['MACD_Alert'] == 1])} | Bearish Count: {len(testing_df[testing_df['MACD_Alert'] == 0])}")
print(f"Bullish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 1])} | Bearish Engulf Count: {len(testing_df[testing_df['Engulf_Alert'] == 0])}")
print(f"Bullish 382 Count: {len(testing_df[testing_df['382_Alert'] == 1])} | Bearish 382 Count: {len(testing_df[testing_df['382_Alert'] == 0])}")
print(f"Bullish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 1])} | Bearish Dual Channel Count: {len(testing_df[testing_df['dual_channel_Alert'] == 0])}")


Bullish MACD Count: 879 | Bearish Count: 995
Bullish Engulf Count: 101 | Bearish Engulf Count: 115
Bullish 382 Count: 1217 | Bearish 382 Count: 1783
Bullish Dual Channel Count: 679 | Bearish Dual Channel Count: 741


## Simulated trading

In [37]:
import pandas as pd

class TradingStrategy:
    def __init__(self, df, initial_capital=10000):
        self.df = df
        self.initial_capital = initial_capital
        self.trades = []
        self.current_trade = {}
        self.total_values = initial_capital

    def execute_trades(self):
        for date in range(len(self.df) - 1):
            try:
                # Sell Trade
                if len(self.current_trade) != 0:
                    if (self.df['dual_channel_Alert'].iloc[date] == 0) or date == len(self.df) - 1:
                        # Has Trade and Bearish
                        self.trades.append(
                            {
                                "Entry_price": self.current_trade["entry_price"],
                                "Entry_date": self.current_trade["entry_date"],
                                "Exit_price": self.df['open'].iloc[date],
                                "Exit_date": self.df['date'].iloc[date],
                                "profit": (self.df['open'].iloc[date] / self.current_trade['entry_price']) - 1,
                                "total_asset": (self.total_values * (self.df.iloc[date].open / self.current_trade['entry_price']) - 1)
                            }
                        )
                        # Update total asset
                        self.total_values = (self.total_values * (self.df.iloc[date + 1].open / self.current_trade['entry_price']))

                        # Close Trade
                        self.current_trade = {}

                # Buy Trade
                elif (self.df['dual_channel_Alert'].iloc[date] == 1) and (len(self.current_trade) == 0):  # No Trade and Bullish
                    self.current_trade["entry_price"] = self.df['close'].iloc[date]
                    self.current_trade["entry_date"] = self.df['date'].iloc[date]

            except IndexError:
                print("No Trade Data")

    def get_trades(self):
        return pd.DataFrame(self.trades)

    def get_total_return(self):
        if self.trades:
            return self.trades[-1]['total_asset']
        return 0

# Usage

strategy = TradingStrategy(testing_df)
strategy.execute_trades()
trades_df = strategy.get_trades()
total_return = strategy.get_total_return()

print(f"Technical Strategic Total Return: {total_return}")

total_return = 0

Technical Strategic Total Return: 606931.0044755108


## Teesting Static Dashboard

In [ ]:
df = collection.find()

# Initialize Dash app
app = dash.Dash(__name__)

# Create figure with subplots
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15, 0.25])

# Define the layout of the Dash app
app.layout = html.Div([
    dcc.Dropdown(
        id='stock-selector',
        options=[{'label': symbol, 'value': symbol} for symbol in collection.distinct("symbol")],
        value=collection.distinct("symbol")[0]
    ),
    dcc.Graph(id='sandbox_testing_graph', figure=fig)
])

# Callback to update the graph based on selected stock and selected range
@app.callback(
    Output('sandbox_testing_graph', 'figure'),
    [Input('stock-selector', 'value'), Input('sandbox_testing_graph', 'relayoutData')]
)
def update_chart(selected_stock, relayout_data):
    # Fetch specific stock
    filtered_query = collection.find({"symbol": f"{selected_stock}"})
    # Convert to pandas dataframe
    filtered_df = pd.DataFrame(list(filtered_query))   
    # Add technical features
    filtered_df = add_features(filtered_df).apply()

    # Add Alerts
    filtered_df = Alert(filtered_df).add_alert()
    
    # Sandbox Testing
    
    trades_history = TradingStrategy(filtered_df)
    trades_history.execute_trades()
    filtered_trades = trades_history.get_trades()
    
    # Create figure with subplots
    fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.65, 0.15, 0.25])

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=filtered_df['date'],
        open=filtered_df['open'],
        high=filtered_df['high'],
        low=filtered_df['low'],
        close=filtered_df['close'],
        name='price'), row=1, col=1)

    # Add EMA traces as lines
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['144EMA'], 
                            mode="lines", name="EMA 144"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['169EMA'],
                            mode="lines", name="EMA 169"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['13EMA'],
                            mode="lines", name="EMA 13"), row=1, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['8EMA'],
                            mode="lines", name="EMA 8"), row=1, col=1)

    # Add Buy / Sell Annotations
    fig.add_trace(go.Scatter(
                    x=filtered_trades.Entry_date,
                    y=filtered_trades.Entry_price,
                    mode="markers",
                    customdata=filtered_trades,
                    marker_symbol="diamond-dot",
                    marker_size=8,
                    marker_line_width=2,
                    marker_line_color="rgba(0,0,0,0.7)",
                    marker_color="rgba(0,255,0,0.7)",
                    hovertemplate="Entry Time: %{customdata[1]}<br>" +\
                        "Entry Price: %{y:.2f}<br>" +\
                        "Total Asset: %{customdata[5]:.3f}",
                    name="Entries"), row=1, col=1)

    fig.add_trace(go.Scatter(
                    x=filtered_trades.Exit_date,
                    y=filtered_trades.Exit_price,
                    mode="markers",
                    customdata=filtered_trades,
                    marker_symbol="diamond-dot",
                    marker_size=8,
                    marker_line_width=2,
                    marker_line_color="rgba(0,0,0,0.7)",
                    marker_color="rgba(255,0,0,0.7)",
                    hovertemplate="Exit Time: %{customdata[1]}<br>" +\
                        "Exit Price: %{y:.2f}<br>" +\
                        "Total Asset: %{customdata[5]:.3f}",
                    name="Exits"), row=1, col=1)

    # Add MACD subplot
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD'], 
                            mode='lines', name='MACD'), row=2, col=1)
    fig.add_trace(go.Scatter(x=filtered_df['date'], y=filtered_df['MACD_SIGNAL'],
                            mode='lines', name='MACD signal'), row=2, col=1)

    # Add Profit subplot
    fig.add_trace(go.Scatter(x=filtered_trades['Exit_date'], y=filtered_trades['total_asset'], 
                            mode='lines', name='Profit'), row=3, col=1)

    # Layout settings
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        autosize=False,
        width=1000,  # Width of the figure in pixels
        height=800,  # Height of the figure in pixels
    )

    # If a new x-axis range is selected or zoomed
    if relayout_data and 'xaxis.range[0]' in relayout_data and 'xaxis.range[1]' in relayout_data:
        x_start = relayout_data['xaxis.range[0]']
        x_end = relayout_data['xaxis.range[1]']

        # Filter the data based on the selected x-axis range
        filtered_data = filtered_df[(filtered_df['date'] >= x_start) & (filtered_df['date'] <= x_end)]
        trades_filtered = filtered_trades[(filtered_trades['Exit_date'] >= x_start) & (filtered_trades['Exit_date'] <= x_end)]
        
        # Calculate the new y-axis range for the candlestick chart
        y_min = filtered_data['low'].min()
        y_max = filtered_data['high'].max()

        # Calculate the new y-axis range for the MACD chart
        y_min_macd = filtered_data[['MACD', 'MACD_SIGNAL']].min().min()
        y_max_macd = filtered_data[['MACD', 'MACD_SIGNAL']].max().max()

        # Calculate the new y-axis range for the Profit chart
        y_min_profit = trades_filtered['total_asset'].min()
        y_max_profit = trades_filtered['total_asset'].max()

        # Update the figure with new y-axis and x-axis ranges
        fig.update_xaxes(range=[x_start, x_end], row=1, col=1)
        fig.update_xaxes(range=[x_start, x_end], row=2, col=1)  
        fig.update_xaxes(range=[x_start, x_end], row=3, col=1) 
        
        fig.update_yaxes(range=[y_min, y_max], row=1, col=1)
        fig.update_yaxes(range=[y_min_macd, y_max_macd], row=2, col=1)
        fig.update_yaxes(range=[y_min_profit, y_max_profit], row=3, col=1)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

## Test Engulfing Method

In [186]:
# Generate random test data
np.random.seed(42)  # For reproducibility

# Define the number of rows for the test DataFrame
num_rows = 100

# Generate random dates
date_rng = pd.date_range(start='2024-09-27 09:30:00', periods=num_rows, freq='5T')

# Generate random data for the DataFrame
test_data = {
    'date': date_rng,
    'symbol': np.random.choice(['AAPL', 'GOOGL', 'MSFT', 'NVDA', 'TSLA'], size=num_rows),
    'interval': np.random.choice(['5m','15m','30m'], size=num_rows),
    '_id': [f'id_{i}' for i in range(num_rows)],
    'datetime': date_rng
}

# Generate random walk for OHLC data
initial_price = 150
price_changes = np.random.normal(loc=0, scale=1, size=num_rows)
prices = initial_price + np.cumsum(price_changes)

test_data['open'] = prices
test_data['high'] = prices + np.random.uniform(low=0, high=5, size=num_rows)
test_data['low'] = prices - np.random.uniform(low=0, high=5, size=num_rows)
test_data['close'] = prices + np.random.uniform(low=-2, high=2, size=num_rows)

# Convert to DataFrame and sort
test_data = pd.DataFrame(test_data).sort_values(by=['symbol','interval','date'])

# Display the test DataFrame
test_data

/var/folders/pv/rbjx7k1j5cs6jxs5pskztclh0000gn/T/ipykernel_29066/2835045034.py:8: FutureWarning:

'T' is deprecated and will be removed in a future version, please use 'min' instead.



,date,symbol,interval,_id,datetime,open,high,low,close
23,2024-09-27 11:25:00,AAPL,15m,id_23,2024-09-27 11:25:00,152.582965,155.239738,148.070201,152.727351
85,2024-09-27 16:35:00,AAPL,15m,id_85,2024-09-27 16:35:00,155.754880,158.482964,152.838036,153.813059
91,2024-09-27 17:05:00,AAPL,15m,id_91,2024-09-27 17:05:00,157.725543,158.072350,152.909431,157.118565
24,2024-09-27 11:30:00,AAPL,30m,id_24,2024-09-27 11:30:00,151.799712,154.502888,149.273450,151.037822
33,2024-09-27 12:15:00,AAPL,30m,id_33,2024-09-27 12:15:00,152.213762,152.606044,147.463453,151.886735
...,...,...,...,...,...,...,...,...,...
62,2024-09-27 14:40:00,TSLA,5m,id_62,2024-09-27 14:40:00,154.089648,156.547728,153.665460,155.905852
71,2024-09-27 15:25:00,TSLA,5m,id_71,2024-09-27 15:25:00,158.018970,161.148269,157.601466,159.108243
72,2024-09-27 15:30:00,TSLA,5m,id_72,2024-09-27 15:30:00,157.795507,160.311188,153.909772,157.876161
80,2024-09-27 16:10:00,TSLA,5m,id_80,2024-09-27 16:10:00,157.172253,161.873404,156.890736,158.192802


In [187]:
import pandas as pd

class CandlePattern:
    def __init__(self, candle: pd.DataFrame):
        self.candle = candle

    def hammer_alert(self):
        # Hammer candle pattern: small body, long lower shadow, little or no upper shadow
        body_size = abs(self.candle['close'] - self.candle['open'])
        lower_shadow = self.candle['low'] - min(self.candle['open'], self.candle['close'])
        upper_shadow = max(self.candle['open'], self.candle['close']) - self.candle['high']
        
        is_hammer = (
            (body_size <= (self.candle['high'] - self.candle['low']) * 0.3) &  # Small body
            (lower_shadow >= (self.candle['high'] - self.candle['low']) * 0.5) &  # Long lower shadow
            (upper_shadow <= (self.candle['high'] - self.candle['low']) * 0.1)  # Little or no upper shadow
        )
        
        date = pd.to_datetime(self.candle['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        return is_hammer, date

    def engulf_alert(self, prev_candle: pd.DataFrame, current_candle: pd.DataFrame):
        # Previous Candle
        prev_open = prev_candle['open']
        prev_close = prev_candle['close']

        # Current Candle
        current_open = current_candle['open']
        current_close = current_candle['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing_prev_red = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open) &
            ((current_close - current_open) > (prev_open - prev_close) * 1.5)  # Current candle significantly larger
        )

        # Bullish Engulfing: current green candle engulfs previous green candle
        bullish_engulfing_prev_green = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open) &
            ((current_close - current_open) > (prev_close - prev_open) * 1.5)  # Current candle significantly larger
        )
        if any([bullish_engulfing_prev_red, bullish_engulfing_prev_green]):
            bullish_engulfing = True
        else:
            bullish_engulfing = False
            
        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )
        date = pd.to_datetime(current_candle['datetime']).strftime('%Y-%m-%d %H:%M:%S')
        
        return bullish_engulfing, bearish_engulfing, date

In [188]:
candle = {}
last_processed_index = {}
pointer_a = 0
pointer_b = 1
alert_data = {}

def streaming_process(data, interval):
    global candle, last_processed_index, pointer_a, pointer_b, alert_data

    # Store the processed index in a dict
    if data['symbol'] not in last_processed_index:
        last_processed_index[data['symbol']] = -1
        last_processed_index['interval'] = interval 
        
    if last_processed_index['interval'] != interval:
        last_processed_index[data['symbol']] = -1
        last_processed_index['interval'] = interval
        pointer_a = 0
        pointer_b = 1
        
    if last_processed_index[data['symbol']] == 0:
        pointer_a = 0
        pointer_b = 1

    # Store the candle data in a dict
    if data['symbol'] not in candle:
        candle[data['symbol']] = []

    candle[data['symbol']].append(data)

    # Initialize alert_data for the symbol if it does not exist
    if data['symbol'] not in alert_data:
        alert_data[data['symbol']] = []

    # Loop through the candle data from the last processed index
    for i in range(last_processed_index[data['symbol']] + 1, len(candle[data['symbol']])):
        candle_data = candle[data['symbol']][i]
        candle_pattern = CandlePattern(candle_data)

        # Check for bullish 0.382 candle patterns
        bullish_382, bullish_382_date = candle_pattern.hammer_alert()

        # Check for engulfing patterns
        if len(candle[data['symbol']]) > 1 and pointer_b < len(candle[data['symbol']]):
            
            pre_candle = candle[data['symbol']][pointer_a]
            curr_candle = candle[data['symbol']][pointer_b]
            
            bullish_engulfing, bearish_engulfing, date = candle_pattern.engulf_alert(pre_candle, curr_candle)

            if bullish_engulfing:
                alert_data[data['symbol']].append({'date': date, 'alert': 'bullish_engulfing'})
                                
            # Update the last processed index
            last_processed_index[data['symbol']] = i
            print(f"symbol: {data['symbol']} comparing {pre_candle['datetime']} and {curr_candle['datetime']} | Processed Index: {last_processed_index[data['symbol']]} | Interval: {last_processed_index['interval']}")
            print(f"pointer a: {pointer_a} | pointer b: {pointer_b}")
            pointer_a += 1
            pointer_b += 1
            break  # Ensure pointers are incremented only once per comparison
            
        
# Test the function with the generated test data
for interval in test_data['interval'].unique():
    for index, row in test_data.iterrows():
        streaming_process(row, interval)

symbol: AAPL comparing 2024-09-27 11:25:00 and 2024-09-27 16:35:00 | Processed Index: 0 | Interval: 15m
pointer a: 0 | pointer b: 1
symbol: AAPL comparing 2024-09-27 11:25:00 and 2024-09-27 16:35:00 | Processed Index: 1 | Interval: 15m
pointer a: 0 | pointer b: 1
symbol: AAPL comparing 2024-09-27 16:35:00 and 2024-09-27 17:05:00 | Processed Index: 2 | Interval: 15m
pointer a: 1 | pointer b: 2
symbol: AAPL comparing 2024-09-27 17:05:00 and 2024-09-27 11:30:00 | Processed Index: 3 | Interval: 15m
pointer a: 2 | pointer b: 3
symbol: AAPL comparing 2024-09-27 11:30:00 and 2024-09-27 12:15:00 | Processed Index: 4 | Interval: 15m
pointer a: 3 | pointer b: 4
symbol: AAPL comparing 2024-09-27 12:15:00 and 2024-09-27 15:50:00 | Processed Index: 5 | Interval: 15m
pointer a: 4 | pointer b: 5
symbol: AAPL comparing 2024-09-27 15:50:00 and 2024-09-27 16:15:00 | Processed Index: 6 | Interval: 15m
pointer a: 5 | pointer b: 6
symbol: AAPL comparing 2024-09-27 16:15:00 and 2024-09-27 16:30:00 | Process

In [189]:
alert_data

{'AAPL': [],
 'GOOGL': [],
 'MSFT': [{'date': '2024-09-27 12:00:00', 'alert': 'bullish_engulfing'}],
 'NVDA': [{'date': '2024-09-27 11:50:00', 'alert': 'bullish_engulfing'},
  {'date': '2024-09-27 11:50:00', 'alert': 'bullish_engulfing'},
  {'date': '2024-09-27 12:50:00', 'alert': 'bullish_engulfing'},
  {'date': '2024-09-27 11:50:00', 'alert': 'bullish_engulfing'},
  {'date': '2024-09-27 12:50:00', 'alert': 'bullish_engulfing'}],
 'TSLA': []}

In [191]:
import pandas as pd
import plotly.graph_objects as go

# Convert alert_data to a DataFrame for easier manipulation
alert_df = pd.DataFrame([(symbol, alert['date'], alert['alert']) for symbol, alerts in alert_data.items() for alert in alerts], columns=['symbol', 'date', 'alert'])

# Convert date column to datetime
alert_df['date'] = pd.to_datetime(alert_df['date'])

# Function to plot candlestick chart with annotations
def plot_candlestick_with_annotations(symbol, data, alerts):
    # Filter data for the selected symbol
    symbol_data = data[data['symbol'] == symbol]
    
    # Create the candlestick chart
    fig = go.Figure(data=[go.Candlestick(
        x=symbol_data['datetime'],
        open=symbol_data['open'],
        high=symbol_data['high'],
        low=symbol_data['low'],
        close=symbol_data['close']
    )])
    
    # Add annotations for bullish engulfing patterns
    for _, alert in alerts.iterrows():
        if alert['alert'] == 'bullish_engulfing':
            fig.add_annotation(
                x=alert['date'],
                y=symbol_data[symbol_data['datetime'] == alert['date']]['close'].values[0],
                text="Bullish Engulfing",
                showarrow=True,
                arrowhead=1
            )
    
    # Update layout
    fig.update_layout(
        title=f'{symbol} Candlestick Chart with Bullish Engulfing Annotations',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False
    )
    
    fig.show()

# Filter alerts for a specific symbol
symbol = 'NVDA'
symbol_alerts = alert_df[alert_df['symbol'] == symbol]

# Plot the candlestick chart with annotations
plot_candlestick_with_annotations(symbol, test_data, symbol_alerts)